<a href="https://colab.research.google.com/github/anantrp/ai-experiments/blob/main/openai_function_calling_stock_prices_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
!pip install openai
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [103]:
global OPENAI_API_KEY
global POLYGON_API_KEY

OPENAI_API_KEY=""
POLYGON_API_KEY=""

openai.api_key = OPENAI_API_KEY

In [154]:
import datetime
import requests
import json
import openai

global today_str

today = datetime.date.today()
today_str = today.strftime('%Y-%m-%d')

def get_stock_price_by_dates(ticker_symbol, date):
    url = f"https://api.polygon.io/v1/open-close/{ticker_symbol}/{date}?adjusted=true&apiKey={POLYGON_API_KEY}"
    try:
        response = requests.request(method='GET', url=url)
        response.raise_for_status()
        # print(response.text)
        return json.dumps(response.text)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def get_stock_sec_details(ticker_symbol, date = today_str):
    url = f"https://api.polygon.io/v3/reference/tickers/{ticker_symbol}?date={date}&apiKey={POLYGON_API_KEY}"
    try:
        response = requests.request(method='GET', url=url)
        response.raise_for_status()
        # print(response.text)
        return json.dumps(response.text)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [145]:
print(get_stock_price_by_dates(ticker_symbol="GOOGL", date="2023-06-15"))

"{\"status\":\"OK\",\"from\":\"2023-06-15\",\"symbol\":\"GOOGL\",\"open\":123.14,\"high\":125.46,\"low\":122.4,\"close\":125.09,\"volume\":3.5146268e+07,\"afterHours\":124.85,\"preMarket\":123.48}"


In [155]:
print(get_stock_sec_details(ticker_symbol="AAPL"))

"{\"request_id\":\"c1f97d28aa630c8c5cb2a37cba71a41f\",\"results\":{\"ticker\":\"AAPL\",\"name\":\"Apple Inc.\",\"market\":\"stocks\",\"locale\":\"us\",\"primary_exchange\":\"XNAS\",\"type\":\"CS\",\"active\":true,\"currency_name\":\"usd\",\"cik\":\"0000320193\",\"composite_figi\":\"BBG000B9XRY4\",\"share_class_figi\":\"BBG001S5N8V8\",\"market_cap\":2.92569585902e+12,\"phone_number\":\"(408) 996-1010\",\"address\":{\"address1\":\"ONE APPLE PARK WAY\",\"city\":\"CUPERTINO\",\"state\":\"CA\",\"postal_code\":\"95014\"},\"description\":\"Apple designs a wide variety of consumer electronic devices, including smartphones (iPhone), tablets (iPad), PCs (Mac), smartwatches (Apple Watch), and AirPods, among others. In addition, Apple offers its customers a variety of services such as Apple Music, iCloud, Apple Care, Apple TV+, Apple Arcade, Apple Fitness, Apple Card, and Apple Pay, among others. Apple's products include internally developed software and semiconductors, and the firm is well known 

In [167]:
def run_conversation(message):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": f"Date today is {today_str}. Provide answer with a date. {message}"}],
        functions=[
            {
                "name": "get_stock_price_by_dates",
                "description": "Get stock prices by dates",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker_symbol": {
                            "type": "string",
                            "description": "The ticker symbol for the stock listed on NASDAQ exchange",
                        },
                        "date": {
                            "type": "string",
                            "description": "date string in %Y-%m-%d format"
                        }
                    },
                    "required": ["stock_ticker", "date"],
                },
            },
            {
                "name": "get_stock_sec_details",
                "description": "Get SEC filing details by stock ticker symbol",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker_symbol": {
                            "type": "string",
                            "description": "The ticker symbol for the stock listed on NASDAQ exchange",
                        },
                        "date": {
                            "type": "string",
                            "description": "date string in %Y-%m-%d format"
                        }
                    },
                    "required": ["stock_ticker"],
                },
            }
        ],
        function_call="auto",
    )

    internal_message = response["choices"][0]["message"]
    # print(internal_message)

    if internal_message.get("function_call"):
        function_name = internal_message["function_call"]["name"]
        function_args = json.loads(internal_message["function_call"]["arguments"])
        function_response = ""

        # Printing which function is detected
        print(function_name)

        if function_name == "get_stock_price_by_dates":
          function_response = get_stock_price_by_dates(
              ticker_symbol=function_args.get("ticker_symbol"),
              date=function_args.get("date"),
          )

        if function_name == "get_stock_sec_details":
          function_response = get_stock_sec_details(
              ticker_symbol=function_args.get("ticker_symbol")
          )

        # print(function_response)
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": message},
                internal_message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

In [168]:
print(run_conversation("tell me apple volume 3 days ago")["choices"][0]["message"]["content"])

get_stock_price_by_dates
The volume for Apple (AAPL) three days ago, on June 13, 2023, was approximately 54,867,129 shares.


In [169]:
print(run_conversation("how many outstanding google shares are there")["choices"][0]["message"]["content"])

get_stock_sec_details
As of now, there are approximately 5,874,000,000 outstanding Google shares.
